## Оценка результатов моделирования

В этом практическом задании три обязательных и две дополнительные задачи. 
<br>
*Обязательные задачи* помогут проверить, что вы действительно усвоили материал. Если у вас всё получилось, можете переходить к следующей теме.
<br>
*Дополнительные задачи* для тех, кто хочет потренироваться в улучшении модели при обнаружении переобучения и в применении кросс-валидации. 
<br>
Удачи!

Цели практического задания: 

1.   Потренироваться в проверке качества на отложенной выборке (выборка test по методике train_validation_test split).
2.   Потренироваться в расчёте метрик с помощью кросс-валидации.
3.   Научиться определять переобучение модели.




В этой практической работе будем использовать датасет Тайваньского центра по переливанию крови. В нём находится следующая информация по донорам:

*   Recency (rec) — количество месяцев, прошедшее с момента последнего донорства;
*   Frequency (freq) — общее количество донорств по донору;
*   Monetary (mon) — количество сданной крови в кубических сантиметрах;
*   Time (time) — количество месяцев, прошедшее с первого донорства;
*   Target (target) — сдал ли донор кровь в марте.

Задача в следующем: предсказать, сдаст ли донор кровь в марте, исходя из его входных параметров.

## Обязательные задачи

In [11]:
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score

In [2]:
df = pd.read_csv('28.2 transfusion_main.csv')
print(df.shape)
df.head()

(648, 5)


,rec,freq,mon,time,target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


**Задача 0. Модель решающего дерева**

1. Изучите предоставленные данные из файла transfusion_main.csv. 
2. Поделите данные на треин/тест. 
3. Обучите модель решающего дерева с random_state=42 на тренировочной выборке. 
4. Сделайте предикт на тестовой выборке. 
5. Посчитайте метрики качества на тренировочной и тестовой выборках. 

Сделайте предположение, переобучена ваша модель или нет. Аргументируйте свой ответ.

In [3]:
# Ваш код здесь
df.dtypes

rec       int64
freq      int64
mon       int64
time      int64
target    int64
dtype: object

In [5]:
# посчитаем и выведем количество пропущенных значений в каждом столбце
pd.isnull(df).sum()

rec       0
freq      0
mon       0
time      0
target    0
dtype: int64

In [6]:
x = df.drop(columns = ['target'], axis = 1) 
y = df.target

In [9]:
y

0      1
1      1
2      1
3      1
4      0
      ..
643    0
644    0
645    0
646    0
647    0
Name: target, Length: 648, dtype: int64

In [12]:
# инициализируем переиенную logreg для обучения модели логистической регрессии
logreg = LogisticRegression(random_state=42)

In [13]:
# импортируем функцию кросс-валидации
cross_validate(logreg, x, y, cv=5)# , scoring=)
# fit_time - время обучения модели для каждой итерации
# score_time - время оценки модели для каждой итерации
# test_score - значение метрики на тестовом фолде

{'fit_time': array([0.01899815, 0.01396251, 0.01296544, 0.01396251, 0.00997424]),
 'score_time': array([0.0009973 , 0.00099754, 0.0009973 , 0.00099683, 0.00099635]),
 'test_score': array([0.81538462, 0.76153846, 0.76923077, 0.80620155, 0.76744186])}

In [14]:
tree = DecisionTreeClassifier(random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
tree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=42)

In [15]:
print(f'качество на треине: {accuracy_score(tree.predict(x_train), y_train)}')
print(f'качество на тесте: {accuracy_score(tree.predict(x_test), y_test)}')

# модель хорошо обучилась на тренировочной выборке, но на тестовой качество просело на две десятых. 
# в целом результат метрики остаётся хорошим, и модель сохраняет свою предиктивную способность. 
# склонность к переобучению у модели есть, но однозначно сказать, что она сильно переобучилась и не может предсказывать новые данные, — нельзя.

качество на треине: 0.9426048565121413
качество на тесте: 0.7384615384615385


**Задача 1. Проверка качества с помощью кросс-валидации**

Проверьте предсказательную способность модели дерева решений с помощью кросс-валидации. Исходя из метрик качества, полученных на кросс-валидации, сделайте вывод о предсказательной способности данной модели.

In [16]:
# Решение Задача 1
cross_val_score(tree, x, y, cv=5)
# как мы видим, метрики качества скачут от итерации к итерации кросс-валидации. 
# модель не показывает стабильных результатов на тестовых фолдах, и поэтому её не рекомендуется использовать в промышленном решении.
# модель с высокой долей вероятности склонна к переобучению. На 1 фолде она показывает очень маленькое значение метрик, что, скорее всего, сигнализирует о переобучении модели.

array([0.50769231, 0.66153846, 0.73846154, 0.72868217, 0.68992248])

**Задача 2. Проверка качества на отложенной выборке**

В результате работы над задачей предсказания сдачи крови вы решили предложить модель дерева решений для использования в промышленном решении. Заказчик хочет проверить качество решения офлайн-тестированием. Для этого он выдал вам выборку данных, которую вы раньше не видели. Вам нужно замерить качество модели на выборке заказчика.

1. Обучите модель деревья решения на всем датасете `df`. 
2. Загрузите отложенную выборку из файла `tranfusioin_oot.csv`. 
3. Проверьте качество модели на этой выборке и на выборке для обучения. 

Сделайте вывод, переобучена ли данная модель.

In [18]:
# Решение Задача 2
oot = pd.read_csv('28.2 transfusion_oot.csv')
oot.head()

,rec,freq,mon,time,target
0,14,1,250,14,0
1,9,9,2250,16,0
2,11,3,750,16,1
3,12,11,2750,23,0
4,2,1,250,2,0


In [19]:
tree.fit(x, y)

df_acc = accuracy_score(tree.predict(x), y)
print(f'качество на df: {df_acc}')
oot_acc = accuracy_score(tree.predict(oot.drop(['target'], axis=1)), oot['target'])
print(f'качество на oot: {oot_acc}')

# модель явно переобучена, потеря в качестве в два раза.

качество на df: 0.941358024691358
качество на oot: 0.54


In [22]:
# дерево строится по одной фиче: rec
for name, imp in zip(x.columns, tree.feature_importances_):
  print(name, imp)

rec 1.0
freq 0.0
mon 0.0
time 0.0


In [23]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [24]:
cross_val_score(tree, x, y, cv=5, scoring='f1_micro')

array([0.76153846, 0.76923077, 0.76923077, 0.76744186, 0.76744186])

In [25]:
cross_val_score(tree, x, y, cv=5, scoring='f1_weighted')

array([0.6584481 , 0.66889632, 0.66889632, 0.66646267, 0.66646267])

## Дополнительные задачи

1. Попробуйте изменить параметры дерева решений таким образом, чтобы модель показывала более стабильные значения метрик на кросс валидации на `df`.

2. Затем проверьте, улучшилась ли предсказательная способности модели на отложенной выборке из датасета `transfusion_oot.csv`.

3. Выведите значимости для каждой фичи из обученного дерева.

**Подсказка:** модель дерева решений склонна к переобучению, так как она стремится строить дерево максимальной глубины. То есть максимально заточить свои ответы под данные, которые она видит в тренировочной выборке. Уменьшить переобучение такой модели можно, настроив параметры, которые отвечают за глубину дерева. Уменьшая глубину обучаемого дерева, вы предотвращаете переобучение. Явление, когда мы искусственно уменьшаем дерево, называется pruning (стрижка дерева).

В случае, если в датасете слишком много фичей, указав слишком маленькую глубину дерева, вы рискуете недообучить модель. 

In [20]:
# Решение Дополнительной задачи
# Мы можем использовать глубину 1, так как у нас всего 4 входные фичи.
tree = DecisionTreeClassifier(random_state=42, max_depth=1)
cross_val_score(tree, x, y, cv=5)

array([0.76153846, 0.76923077, 0.76923077, 0.76744186, 0.76744186])

In [21]:
tree.fit(x, y)

df_acc = accuracy_score(tree.predict(x), y)
print(f'качество на df: {df_acc}')
oot_acc = accuracy_score(tree.predict(oot.drop(['target'], axis=1)), oot['target'])
print(f'качество на oot: {oot_acc}')

# Качество модели на df и отложенной выборке не сильно отличается, модель не переобучилась.

качество на df: 0.7669753086419753
качество на oot: 0.73


In [ ]:
# Решение Задача 0
x = df.drop(['target'], axis=1)
y = df.target

In [ ]:
tree = DecisionTreeClassifier(random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
tree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
print(f'качество на треине: {accuracy_score(tree.predict(x_train), y_train)}')
print(f'качество на тесте: {accuracy_score(tree.predict(x_test), y_test)}')

# модель хорошо обучилась на тренировочной выборке, но на тестовой качество просело на две десятых. 
# в целом результат метрики остаётся хорошим, и модель сохраняет свою предиктивную способность. 
# склонность к переобучению у модели есть, но однозначно сказать, что она сильно переобучилась и не может предсказывать новые данные, — нельзя.

качество на треине: 0.9426048565121413
качество на тесте: 0.7384615384615385


In [ ]:
# Решение Задача 1
cross_val_score(tree, x, y, cv=5)
# как мы видим, метрики качества скачут от итерации к итерации кросс-валидации. 
# модель не показывает стабильных результатов на тестовых фолдах, и поэтому её не рекомендуется использовать в промышленном решении.
# модель с высокой долей вероятности склонна к переобучению. На 1 фолде она показывает очень маленькое значение метрик, что, скорее всего, сигнализирует о переобучении модели.

array([0.50769231, 0.66153846, 0.73846154, 0.72868217, 0.68992248])

In [ ]:
# Решение Задача 2
oot = pd.read_csv('transfusion_oot.csv')
oot.head()

,rec,freq,mon,time,target
0,14,1,250,14,0
1,9,9,2250,16,0
2,11,3,750,16,1
3,12,11,2750,23,0
4,2,1,250,2,0


In [ ]:
tree.fit(x, y)

df_acc = accuracy_score(tree.predict(x), y)
print(f'качество на df: {df_acc}')
oot_acc = accuracy_score(tree.predict(oot.drop(['target'], axis=1)), oot['target'])
print(f'качество на oot: {oot_acc}')

# модель явно переобучена, потеря в качестве в два раза.

качество на df: 0.941358024691358
качество на oot: 0.54


In [ ]:
# Решение Дополнительной задачи
# Мы можем использовать глубину 1, так как у нас всего 4 входные фичи.
tree = DecisionTreeClassifier(random_state=42, max_depth=1)
cross_val_score(tree, x, y, cv=5)

array([0.76153846, 0.76923077, 0.76923077, 0.76744186, 0.76744186])

In [ ]:
tree.fit(x, y)

df_acc = accuracy_score(tree.predict(x), y)
print(f'качество на df: {df_acc}')
oot_acc = accuracy_score(tree.predict(oot.drop(['target'], axis=1)), oot['target'])
print(f'качество на oot: {oot_acc}')

# Качество модели на df и отложенной выборке не сильно отличается, модель не переобучилась.

качество на df: 0.7669753086419753
качество на oot: 0.73


In [ ]:
# дерево строится по одной фиче: rec
for name, imp in zip(x.columns, tree.feature_importances_):
  print(name, imp)

rec 1.0
freq 0.0
mon 0.0
time 0.0


Попробуйте подставить другие метрики для рассчёта кросс-валидации с помощью параметра `scoring`. Чтобы узнать список метрик, которые может принимать кросс-валидация, вызовите метод `sklearn.metrics.SCORERS.keys()`.
Заметьте, в этом списке есть не только метрики классификации, но и регрессии. Все они нам не подойдут.

Рассчитайте значение кросс-валидации на метрике f1 micro, f1 weighted.


In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [ ]:
cross_val_score(tree, x, y, cv=5, scoring='f1_micro')

array([0.76153846, 0.76923077, 0.76923077, 0.76744186, 0.76744186])

In [ ]:
cross_val_score(tree, x, y, cv=5, scoring='f1_weighted')

array([0.6584481 , 0.66889632, 0.66889632, 0.66646267, 0.66646267])